In [1]:
from flask import Flask
from flask import request, jsonify
import base64
from bs4 import BeautifulSoup
from pymystem3 import Mystem
import re
import nltk
import pickle

In [2]:
wv_model = pickle.load(open('w2v_model.pickle', 'rb'))
rf = pickle.load(open('rf_model.pickle', 'rb'))

In [3]:
nltk.download('punkt')
morph = Mystem()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/antonina.goryacheva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
def text_to_sent(t):
    text = BeautifulSoup(t).text.lower()
    tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')
    raw_sentences = tokenizer.tokenize(text.strip())
    raw_sentences = [x.split(';') for x in raw_sentences]
    raw_sentences = sum(raw_sentences, [])
    sentences = [morph.lemmatize(x) for x in raw_sentences]
    return [[y for y in x if re.match('[а-яёa-z0-9]', y)]
            for x in sentences]

def text_to_vec(words, model):
    index2word_set = set(model.wv.index2word)
    text_vec = np.zeros((300,), dtype="float32")
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model.wv[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec

In [7]:
import pandas as pd

In [8]:
test = pd.read_csv('test.csv', sep=';')

In [9]:
test.head()

,id,text
0,31063,<p><strong>В крупную компанию по организации и...
1,31064,<p><strong>Обязанности:</strong></p> <ul> <li>...
2,31065,<p> </p> <p><strong>Обязанности:</strong></p> ...
3,31066,<p><strong>Обязанности:</strong></p> <ul> <li>...
4,31067,<p><strong>Вакансия СРОЧНАЯ!</strong></p> <p><...


In [12]:
import numpy as np

In [19]:
text = test['text'][0]
s = text_to_sent(text)
s = sum(s, [])

In [25]:
vec.shape[0]

300

In [29]:
vec = text_to_vec(s, wv_model)

resp = {
    'predict': rf.predict(vec.reshape((1, vec.shape[0])))[0],
}

In [32]:
text

'<p><strong>В крупную компанию по организации и приготовлению корпоративного питания требуется - Бухгалтер калькулятор.</strong></p> <p><strong>Обязанности:</strong></p> <ul> <li>работа с технологическими картами</li> <li>расчет себестоимости блюд</li> <li>ведение номенклатур</li> <li>ведение документооборота производства</li> <li>ценообразование</li> <li>участие в проведении инвентаризации</li> </ul> <strong>Требования:</strong> <ul> <li>опыт работы в общепите бухгалтером-калькулятором</li> <li>умение работать в программе общепит &quot;Айко&quot; (или другой)</li> <li>внимательность и ответственное отношение к работе</li> </ul> <strong>Условия:</strong> <ul> <li>официальное трудоустройство с первого рабочего дня</li> <li>место работы Советский район</li> <li>график работы 5/2 с 9:00 до 18:00</li> <li>предоставляется бесплатное питание</li> </ul>'

In [34]:
import requests

resp = requests.post(
    'http://localhost:11010/predict',
    data = {
        'text': [text]
    }
)

In [36]:
resp.text

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request.  Either the server is overloaded or there is an error in the application.</p>\n'